In [1]:
import pandas as pd

In [47]:
df_enrollment = pd.read_csv('data/csv/enrollment.csv')

In [48]:
df_enrollment.head()

,Unnamed: 0,student_id,year,semester,course_id
0,0,0,2006-2007,Master semestre 2,2
1,1,0,2006-2007,Master semestre 1,754
2,2,0,2006-2007,Master semestre 1,1038
3,3,0,2006-2007,Master semestre 2,4693
4,4,0,2006-2007,Master semestre 1,8632


In [49]:
len(df_enrollment)

831129

In [50]:
df_enrollment.dropna(axis=0,how='any',inplace=True)

In [51]:
len(df_enrollment)

823143

In [52]:
df_enrollment['year'].unique()

array(['2006-2007', '2010-2011', '2007-2008', '2012-2013', '2015-2016',
       '2016-2017', '2014-2015', '2013-2014', '2017-2018', '2018-2019',
       '2019-2020', '2011-2012', '2009-2010', '2008-2009', '2005-2006',
       '2004-2005', 'technologie et entrepreneuriat',
       'architecture et durabilité',
       'contrôle et systèmes intelligents (edoc)',
       'Chimie et génie chimique (edoc)', 'Intersemestre été',
       'Semestre automne', 'Semestre printemps'], dtype=object)

In [53]:
df_clean_enrollment = df_enrollment[df_enrollment['year'].str.contains('20')]

In [54]:
df_clean_enrollment['year'].unique()

array(['2006-2007', '2010-2011', '2007-2008', '2012-2013', '2015-2016',
       '2016-2017', '2014-2015', '2013-2014', '2017-2018', '2018-2019',
       '2019-2020', '2011-2012', '2009-2010', '2008-2009', '2005-2006',
       '2004-2005'], dtype=object)

In [55]:
df_clean_enrollment['semester'].unique()

array([' Master semestre 2', ' Master semestre 1', ' Master semestre 4',
       ' Master semestre 3', ' Bachelor semestre 3',
       ' Bachelor semestre 5', ' Bachelor semestre 4',
       ' Bachelor semestre 2', ' Bachelor semestre 1',
       ' Bachelor semestre 6', ' Semestre automne', ' Semestre printemps',
       ' Bachelor semestre 5b', ' Bachelor semestre 6b',
       ' Mineur semestre 2', ' Mineur semestre 1'], dtype=object)

In [56]:
df_clean_enrollment = df_clean_enrollment[df_clean_enrollment['semester'].str.contains('Master')]

In [57]:
df_clean_enrollment['semester'].unique()

array([' Master semestre 2', ' Master semestre 1', ' Master semestre 4',
       ' Master semestre 3'], dtype=object)

In [58]:
len(df_clean_enrollment)

303286

In [59]:
df_course = pd.read_csv('data/csv/courses.csv')

In [60]:
df_course.head()

,course_id,course_name,year
0,1,Biological and physiological transport,2006-2007
1,2,Biological and physiological transport,2007-2008
2,3,Biological and physiological transport,2007-2008
3,4,Special topics in reactor physics,2015-2016
4,5,Special topics in reactor physics,2014-2015


In [61]:
df_student = pd.read_csv('data/csv/student.csv')

In [62]:
df_student.head()

,Unnamed: 0,student_name,section,student_id
0,0,Bays Emmanuelle,Bioingénierie,0
1,1,Berlier Guillaume,Bioingénierie,1
2,2,Cosson Steffen,Bioingénierie,2
3,3,De Pablo Pena Albora,Bioingénierie,3
4,4,de Valence Sarra,Bioingénierie,4


In [63]:
df_enrollment_course = df_clean_enrollment.merge(df_course, left_on = 'course_id', right_on ='course_id', how='inner')

In [68]:
df_enrollment_course

,Unnamed: 0,student_id,student_year,semester,course_id,course_name,course_year
0,0,0,2006-2007,Master semestre 2,2,Biological and physiological transport,2007-2008
1,18,1,2006-2007,Master semestre 2,2,Biological and physiological transport,2007-2008
2,47,2,2006-2007,Master semestre 2,2,Biological and physiological transport,2007-2008
3,66,3,2006-2007,Master semestre 2,2,Biological and physiological transport,2007-2008
4,74,4,2006-2007,Master semestre 2,2,Biological and physiological transport,2007-2008
...,...,...,...,...,...,...,...
303269,830122,813545,2017-2018,Master semestre 2,40233,Hydrodynamics,2015-2016
303270,830507,820975,2012-2013,Master semestre 1,40429,Advanced multiprocessor architecture,2013-2014
303271,830508,820976,2012-2013,Master semestre 1,40429,Advanced multiprocessor architecture,2013-2014
303272,830509,820977,2013-2014,Master semestre 3,40431,Advanced multiprocessor architecture,2013-2014


In [75]:
df_enrollment_course.dtypes

Unnamed: 0       int64
student_id       int64
student_year    object
semester        object
course_id        int64
course_name     object
course_year     object
dtype: object

In [67]:
df_enrollment_course.rename(columns={"year_x": "student_year", "year_y":"course_year"},inplace =True)

In [82]:
df_clean_enrollment.groupby('year').count()

,Unnamed: 0,student_id,semester,course_id
year,,,,
2004-2005,11872,11872,11872,11872
2005-2006,14648,14648,14648,14648
2006-2007,16130,16130,16130,16130
2007-2008,14241,14241,14241,14241
2008-2009,13833,13833,13833,13833
2009-2010,14940,14940,14940,14940
2010-2011,15629,15629,15629,15629
2011-2012,18117,18117,18117,18117
2012-2013,19554,19554,19554,19554


In [ ]:
df_clean_enrollment = 